# Used Car Price Prediction Project 

In [35]:
import numpy as np
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt

In [36]:
df=pd.read_csv('usedCars.csv')
df.head()

,Id,Company,Model,Variant,FuelType,Colour,Kilometer,BodyStyle,TransmissionType,ManufactureDate,ModelYear,CngKit,Price,Owner,DealerState,DealerName,City,Warranty,QualityScore
0,555675,MARUTI SUZUKI,CELERIO(2017-2019),1.0 ZXI AMT O,PETROL,Silver,33197,HATCHBACK,NaN,2018-02-01,2018,NaN,5.75 Lakhs,1st Owner,Karnataka,Top Gear Cars,Bangalore,1,7.8
1,556383,MARUTI SUZUKI,ALTO,LXI,PETROL,Red,10322,HATCHBACK,Manual,2021-03-01,2021,NaN,4.35 Lakhs,1st Owner,Karnataka,Renew 4 u Automobiles PVT Ltd,Bangalore,1,8.3
2,556422,HYUNDAI,GRAND I10,1.2 KAPPA ASTA,PETROL,Grey,37889,HATCHBACK,Manual,2015-03-01,2015,NaN,4.7 Lakhs,1st Owner,Karnataka,Anant Cars Auto Pvt Ltd,Bangalore,1,7.9
3,556771,TATA,NEXON,XT PLUS,PETROL,A Blue,13106,HATCHBACK,NaN,2020-08-01,2020,NaN,9.9 Lakhs,1st Owner,Karnataka,Adeep Motors,Bangalore,1,8.1
4,559619,FORD,FIGO,EXI DURATORQ 1.4,DIESEL,Silver,104614,HATCHBACK,Manual,2010-11-01,2010,NaN,2.7 Lakhs,2nd Owner,Karnataka,Zippy Automart,Bangalore,0,7.5


In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1064 entries, 0 to 1063
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Id                1064 non-null   int64  
 1   Company           1064 non-null   object 
 2   Model             1064 non-null   object 
 3   Variant           1064 non-null   object 
 4   FuelType          1063 non-null   object 
 5   Colour            1064 non-null   object 
 6   Kilometer         1064 non-null   int64  
 7   BodyStyle         1064 non-null   object 
 8   TransmissionType  350 non-null    object 
 9   ManufactureDate   1064 non-null   object 
 10  ModelYear         1064 non-null   int64  
 11  CngKit            22 non-null     object 
 12  Price             1064 non-null   object 
 13  Owner             1064 non-null   object 
 14  DealerState       1064 non-null   object 
 15  DealerName        1064 non-null   object 
 16  City              1064 non-null   object 


In [38]:
df['Price'].sort_values()

992       1 Lakhs
953    1.19 Lakhs
976     1.2 Lakhs
917     1.4 Lakhs
508    1.45 Lakhs
          ...    
179    9.91 Lakhs
749    9.92 Lakhs
724    9.97 Lakhs
5      9.99 Lakhs
291        95,000
Name: Price, Length: 1064, dtype: object

**Type Casting Price column to Float**

In [39]:
def convert_amnt(amt):
    if 'Lakhs' in amt:
        # 1 Lakhs=100000 thats why conversion is done 
        return float(amt.replace(' Lakhs','').replace(',',''))*100000
    else:
        # there is some entries which contain comma( , )
        return float(amt.replace(',',''))
df['Price']=df['Price'].apply(convert_amnt)

In [40]:
df.describe(include='all')

,Id,Company,Model,Variant,FuelType,Colour,Kilometer,BodyStyle,TransmissionType,ManufactureDate,ModelYear,CngKit,Price,Owner,DealerState,DealerName,City,Warranty,QualityScore
count,1064.000000,1064,1064,1064,1063,1064,1064.000000,1064,350,1064,1064.000000,22,1.064000e+03,1064,1064,1064,1064,1064.000000,1064.000000
unique,NaN,23,218,575,5,76,NaN,10,9,162,NaN,2,NaN,4,10,57,11,NaN,NaN
top,NaN,MARUTI SUZUKI,SWIFT,VXI,PETROL,White,NaN,HATCHBACK,Manual,2018-01-01,NaN,Company Fitted,NaN,1st Owner,Delhi,Car Choice Exclusif,Delhi,NaN,NaN
freq,NaN,252,38,37,670,289,NaN,423,269,25,NaN,14,NaN,897,196,67,196,NaN,NaN
mean,568156.542293,NaN,NaN,NaN,NaN,NaN,52807.187970,NaN,NaN,NaN,2016.864662,NaN,8.350536e+05,NaN,NaN,NaN,NaN,0.738722,7.770207
std,16438.139974,NaN,NaN,NaN,NaN,NaN,33840.296979,NaN,NaN,NaN,2.996786,NaN,5.726538e+05,NaN,NaN,NaN,NaN,0.439538,0.719717
min,525978.000000,NaN,NaN,NaN,NaN,NaN,101.000000,NaN,NaN,NaN,2003.000000,NaN,9.500000e+04,NaN,NaN,NaN,NaN,0.000000,0.000000
25%,555321.500000,NaN,NaN,NaN,NaN,NaN,32113.500000,NaN,NaN,NaN,2015.000000,NaN,4.850000e+05,NaN,NaN,NaN,NaN,0.000000,7.500000
50%,572753.000000,NaN,NaN,NaN,NaN,NaN,49432.000000,NaN,NaN,NaN,2017.000000,NaN,6.750000e+05,NaN,NaN,NaN,NaN,1.000000,7.800000
75%,583072.750000,NaN,NaN,NaN,NaN,NaN,68828.500000,NaN,NaN,NaN,2019.000000,NaN,9.850000e+05,NaN,NaN,NaN,NaN,1.000000,8.100000


In [41]:
#Null values percentage wise
df.isnull().sum()/len(df)*100

Id                   0.000000
Company              0.000000
Model                0.000000
Variant              0.000000
FuelType             0.093985
Colour               0.000000
Kilometer            0.000000
BodyStyle            0.000000
TransmissionType    67.105263
ManufactureDate      0.000000
ModelYear            0.000000
CngKit              97.932331
Price                0.000000
Owner                0.000000
DealerState          0.000000
DealerName           0.000000
City                 0.000000
Warranty             0.000000
QualityScore         0.000000
dtype: float64

**In this dataset - three columns have missing values -FuelType,TransmissionType and CngKit.<br> Will remove the CngKit column because CNG cars can be identified from the FuelType column<br>In TransmissionType, 67% data is missing lets drop this column.<br>And in FuelType will drop the rows with null values**

In [44]:
df.drop('CngKit',axis=1,inplace=True)
df.drop('TransmissionType',axis=1,inplace=True)
#Removing null values from FuelType column
df['FuelType'].dropna(inplace=True)

In [60]:
df['Variant'].nunique()

575

**The number of unique variants is very high and would create too many categories (resulting in overfitting or sparsity), it might be better to drop it**

In [ ]:
#df.drop('Variant',axis=1,inplace=True)

In [61]:
df[['ManufactureDate','ModelYear']]

,ManufactureDate,ModelYear
0,2018-02-01,2018
1,2021-03-01,2021
2,2015-03-01,2015
3,2020-08-01,2020
4,2010-11-01,2010
...,...,...
1059,2020-09-01,2020
1060,2019-01-01,2019
1061,2013-05-01,2013
1062,2018-06-01,2018


**Dropping ManufactureDate column as it the age of the car and we already have the ModelYear column**

In [67]:
df.drop('ManufactureDate',axis=1,inplace=True)

In [ ]:
#dropping column ID, as it is a identifier and not required for analysis
df.drop('Id',axis=1,inplace=True)

In [63]:
df['Model'].value_counts()

Model
SWIFT                   38
CITY                    38
BALENO                  37
CRETA                   30
ECOSPORT                28
                        ..
S-CROSS(2017-2020)       1
COMPASS(2020-2021)       1
ELITE I20(2020-2020)     1
XCENT(2017-2020)         1
SCORPIO(2009-2014)       1
Name: count, Length: 218, dtype: int64

**Changing the model year column to car age column**

In [81]:
df['ModelYear'] = 2023 - df['ModelYear']
df.rename(columns={'ModelYear':'Age'},inplace=True)

In [82]:
df.select_dtypes(include=['int64','float64']).columns

Index(['Kilometer', 'Age', 'Price', 'Warranty', 'QualityScore'], dtype='object')